# Analysing Data

<hr style="border:2px solid gray">

## Index: <a id='index'></a>

1. [Interpolation](#i)
    1. [Linear Interpolation](#li)
    1. [Cubic Spline](#cs)
1. [iminuit](#im)
1. [Using iminuit](#ui)
    1. [The Basics](#tb)
    1. [Gradient Descent](#gd)
    1. [The Minuit object](#tmo)
    1. [Parameter Uncertainties, Covariances, and Confidence Intervals](#pucaci)
    1. [Maximum-likelihood fits](#mlf)